# <div align="center"> Oskar Borkowski <div align="center"> Lab 3   

## Łączenie tabel, podzapytania i funkcje agregujące

## Zadanie wprowadzające

In [1]:
import main
import pandas as pd
import psycopg2 as pg
c = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020');
pd.set_option("display.max_rows", 20, "display.max_columns", 8)


In [2]:
print("\n1. ilość filmów tej samej długości")
df = pd.read_sql('SELECT length, COUNT(length) "amount" FROM film \
                GROUP BY length ORDER BY length ASC', c)
df


1. ilość filmów tej samej długości


/tmp/ipykernel_70141/2357963750.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT length, COUNT(length) "amount" FROM film \


,length,amount
0,46,5
1,47,7
2,48,11
3,49,5
4,50,9
...,...,...
135,181,10
136,182,6
137,183,5
138,184,8


In [3]:
print("\n2. klienci z tego samego miasta")
df = pd.read_sql('SELECT city.city, COUNT(customer_id) "clients amount" FROM customer \
                INNER JOIN address ON customer.address_id = address.address_id \
                INNER JOIN city ON address.city_id = city.city_id \
                GROUP BY city \
                ORDER BY COUNT(customer_id) DESC', c)
df


2. klienci z tego samego miasta


/tmp/ipykernel_70141/3603048380.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT city.city, COUNT(customer_id) "clients amount" FROM customer \


,city,clients amount
0,London,2
1,Aurora,2
2,Tokat,1
3,Atlixco,1
4,Mukateve,1
...,...,...
592,Greensboro,1
593,Ocumare del Tuy,1
594,Southport,1
595,Pemalang,1


In [4]:
print("\n3. średni koszt wypożyczenia")
df = pd.read_sql('SELECT AVG(amount) FROM payment', c)
df

/tmp/ipykernel_70141/1482322744.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT AVG(amount) FROM payment', c)



3. średni koszt wypożyczenia


,avg
0,4.200606


In [5]:
print("\n4. ilość filmów danej kategori")
df = pd.read_sql('SELECT name "genre", COUNT(film.title) "amount" FROM category \
                INNER JOIN film_category ON category.category_id = film_category.category_id \
                INNER JOIN film ON film_category.film_id = film.film_id \
                GROUP BY category.name \
                ORDER BY COUNT(title) DESC', c)
df


4. ilość filmów danej kategori


/tmp/ipykernel_70141/1681996814.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT name "genre", COUNT(film.title) "amount" FROM category \


,genre,amount
0,Sports,74
1,Foreign,73
2,Family,69
3,Documentary,68
4,Animation,66
5,Action,64
6,New,63
7,Drama,62
8,Sci-Fi,61
9,Games,61


In [6]:
print("\n5. liczba klientów danego kraju")
df = pd.read_sql('SELECT country, COUNT(customer.customer_id) "clients amount" FROM country \
                INNER JOIN city ON country.country_id = city.country_id \
                INNER JOIN address ON city.city_id = address.city_id \
                INNER JOIN customer ON address.address_id = customer.address_id \
                GROUP BY country \
                ORDER BY COUNT(customer_id) DESC', c)
df


5. liczba klientów danego kraju


/tmp/ipykernel_70141/417740757.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT country, COUNT(customer.customer_id) "clients amount" FROM country \


,country,clients amount
0,India,60
1,China,53
2,United States,36
3,Japan,31
4,Mexico,30
...,...,...
103,Moldova,1
104,Tonga,1
105,Nepal,1
106,Greenland,1


In [7]:
print("\n6. sklepy z liczbą klientów 100 < k < 300")
df = pd.read_sql('SELECT store.store_id "store id", address.address, country.country, \
                staff.first_name "manager CN", staff.last_name "manager SN", \
                count(customer.customer_id) "customer amount" FROM store \
                INNER JOIN customer ON store.store_id = customer.store_id \
                INNER JOIN staff ON store.manager_staff_id = staff.staff_id \
                INNER JOIN address ON store.address_id = address.address_id \
                INNER JOIN city ON address.city_id = city.city_id \
                INNER JOIN country ON country.country_id = city.country_id \
                GROUP by store.store_id, address.address, country.country, staff.first_name, staff.last_name \
                HAVING COUNT(customer.customer_id) > 100 AND COUNT(customer.customer_id) < 300', c)
df


6. sklepy z liczbą klientów 100 < k < 300


/tmp/ipykernel_70141/912230140.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT store.store_id "store id", address.address, country.country, \


,store id,address,country,manager CN,manager SN,customer amount
0,2,28 MySQL Boulevard,Australia,Jon,Stephens,273


In [8]:
print("\n7. klienci z łączynm czasem oglądania powyżej 200h")
df = pd.read_sql('SELECT first_name, last_name, SUM(film.length)/60 "watch time" FROM customer \
                INNER JOIN rental ON rental.customer_id = customer.customer_id \
                INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id \
                INNER JOIN film ON inventory.film_id = film.film_id \
                GROUP BY customer.first_name, customer.last_name \
                HAVING SUM(film.length) > 200*60 \
                ORDER BY SUM(film.length) DESC', c)
df


7. klienci z łączynm czasem oglądania powyżej 200h


/tmp/ipykernel_70141/801966612.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT first_name, last_name, SUM(film.length)/60 "watch time" FROM customer \


,first_name,last_name,watch time


In [9]:
print("\n8. średnia wartość filmu")
df = pd.read_sql('SELECT AVG(rental_rate) FROM film', c)
df


8. średnia wartość filmu


/tmp/ipykernel_70141/2117113983.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT AVG(rental_rate) FROM film', c)


,avg
0,2.98


In [10]:
print("\n9. średnia długość filmu danej katergori")
df = pd.read_sql('SELECT name AS genre, AVG(film.length) AS average FROM category \
                INNER JOIN film_category ON category.category_id = film_category.category_id \
                INNER JOIN film ON film_category.film_id = film.film_id \
                GROUP BY name ORDER BY average DESC', c)
df


9. średnia długość filmu danej katergori


/tmp/ipykernel_70141/2012698639.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT name AS genre, AVG(film.length) AS average FROM category \


,genre,average
0,Sports,128.202703
1,Games,127.836066
2,Foreign,121.698630
3,Drama,120.838710
4,Comedy,115.827586
5,Family,114.782609
6,Music,113.647059
7,Travel,113.315789
8,Horror,112.482143
9,Classics,111.666667


In [11]:
print("\n10. najdłuższy tytuł w danej katergori")
df = pd.read_sql('SELECT name AS genre, MAX(length(film.title)) from category \
                INNER JOIN film_category ON category.category_id = film_category.category_id \
                INNER JOIN film ON film_category.film_id = film.film_id \
                GROUP BY name', c)
df


10. najdłuższy tytuł w danej katergori


/tmp/ipykernel_70141/3123529029.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT name AS genre, MAX(length(film.title)) from category \


,genre,max
0,Sports,25
1,Classics,23
2,New,21
3,Family,22
4,Comedy,23
5,Animation,22
6,Travel,23
7,Music,22
8,Drama,22
9,Horror,27


In [12]:
print("\n11. najdłuższy film danej katergori")
df = pd.read_sql('SELECT name AS genre, MAX(film.length) AS "longest - total minutes" from category \
                INNER JOIN film_category ON category.category_id = film_category.category_id \
                INNER JOIN film ON film_category.film_id = film.film_id \
                GROUP BY category.name ORDER BY "longest - total minutes" DESC', c)
df


11. najdłuższy film danej katergori


/tmp/ipykernel_70141/2893161874.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT name AS genre, MAX(film.length) AS "longest - total minutes" from category \


,genre,longest - total minutes
0,Music,185
1,Games,185
2,Action,185
3,Sci-Fi,185
4,Comedy,185
5,Animation,185
6,Travel,185
7,Foreign,184
8,Family,184
9,Sports,184


## Zadanie implementacyjne

In [13]:
main.film_in_category(5)

/media/loren/Documents/AGH/SEM5/AiBD/AiBD_repo/Lab3/main.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df =  pd.read_sql_query("SELECT title, language.name, category.name FROM film \


,title,name,name
0,Airplane Sierra,English,Comedy
1,Anthem Luke,English,Comedy
2,Bringing Hysterical,English,Comedy
3,Caper Motions,English,Comedy
4,Cat Coneheads,English,Comedy
...,...,...,...
53,Valley Packer,English,Comedy
54,Velvet Terminator,English,Comedy
55,Vertigo Northwest,English,Comedy
56,Wisdom Worker,English,Comedy


In [14]:
main.number_films_in_category(5)

/media/loren/Documents/AGH/SEM5/AiBD/AiBD_repo/Lab3/main.py:53: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df =  pd.read_sql_query("SELECT name, COUNT(film_category.film_id) FROM category \


,name,count
0,Comedy,58


In [15]:
main.number_film_by_length(49,59)

/media/loren/Documents/AGH/SEM5/AiBD/AiBD_repo/Lab3/main.py:79: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df =  pd.read_sql_query("SELECT length, COUNT(title)FROM film \


,length,count
0,49,5
1,50,9
2,51,7
3,52,7
4,53,9
5,54,6
6,55,2
7,56,5
8,57,7
9,58,7


In [16]:
main.client_from_city("London")

/media/loren/Documents/AGH/SEM5/AiBD/AiBD_repo/Lab3/main.py:104: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df =  pd.read_sql_query("SELECT city, customer.first_name, customer.last_name FROM city \


,city,first_name,last_name
0,London,Cecil,Vines
1,London,Mattie,Hoffman


In [17]:
main.avg_amount_by_length(55)

/media/loren/Documents/AGH/SEM5/AiBD/AiBD_repo/Lab3/main.py:128: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df =  pd.read_sql_query("SELECT length, AVG(payment.amount) FROM film \


,length,avg
0,55,1.605385


In [18]:
main.client_by_sum_length(1934)

/media/loren/Documents/AGH/SEM5/AiBD/AiBD_repo/Lab3/main.py:153: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df =  pd.read_sql_query("SELECT first_name, last_name, SUM(film.length) FROM customer \


,first_name,last_name,sum
0,Aaron,Selby,2691
1,Adam,Gooch,2437
2,Adrian,Clary,2483
3,Agnes,Bishop,2741
4,Alan,Kahn,3059
...,...,...,...
581,Willie,Markham,2887
582,Wilma,Richards,2641
583,Yolanda,Weaver,2946
584,Yvonne,Watkins,2513


In [19]:
main.category_statistic_length("Comedy")

/media/loren/Documents/AGH/SEM5/AiBD/AiBD_repo/Lab3/main.py:179: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df =  pd.read_sql_query("SELECT name, AVG(film.length), SUM(film.length), MAX(film.length), MIN(film.length) FROM category \


,name,avg,sum,max,min
0,Comedy,115.827586,6718,185,47
